# TidyData: Concatenation

This section details how you can concatenate multiple tidy data sheets together when you output them.

## Source Data

The data source we're using for these examples is shown below:

| <span style="color:green">Note - this particular table has some very verbose headers we don't care about, so we'll be using `bounded=` to remove them from the previews as well as to show just the subset of data we're working with.</span>|
|-----------------------------------------|

The [full data source can be downloaded here](https://github.com/mikeAdamss/tidychef/raw/main/tests/fixtures/xlsx/ons-oic.xlsx). We'll be using th 10th tab named "Table 3c".

In [ ]:
from tidychef import acquire, preview
from tidychef.selection import XlsxSelectable

table: XlsxSelectable = acquire.xlsx.http("https://github.com/mikeAdamss/tidychef/raw/main/tests/fixtures/xlsx/ons-oic.xlsx", tables="Table 3c")
preview(table, bounded="A4:H10")

## The "Two" Sources

To keep these examples reasonably brief we're going to create two TidyData class from the above as follow:

- one with observations from B7 though H8
- one with observations from B9 through H10

In [ ]:
from tidychef import acquire, preview
from tidychef.direction import right, down
from tidychef.output import TidyData, Column
from tidychef.selection import XlsxSelectable

table: XlsxSelectable = acquire.xlsx.http("https://github.com/mikeAdamss/tidychef/raw/main/tests/fixtures/xlsx/ons-oic.xlsx", tables="Table 3c")


houses = table.excel_ref("B5").expand(right).label_as("Houses")
dataset_identifier_code = houses.shift(down).label_as("Dataset Identifier Code")
period = table.excel_ref("A7").expand(down).label_as("Period")

observations1 = table.excel_ref("B7:H8").label_as("Observations")
observations2 = table.excel_ref("B9:H10").label_as("Observations")

In [ ]:
## Preview Both Sets of Selection

print("Preview selections for tidydata1")
preview(observations1, houses, dataset_identifier_code, period, bounded="A4:H10")

print("Preview selections for tidydata1")
preview(observations2, houses, dataset_identifier_code, period, bounded="A4:H10")

## Create and preview TidyData objects

In [ ]:
tidydata1 = TidyData(
    observations1,
    Column(period.finds_observations_directly(right)),
    Column(dataset_identifier_code.finds_observations_directly(down)),
    Column(houses.finds_observations_directly(down))
)
print("Preview tidydata1")
print(tidydata1)

tidydata2 = TidyData(
    observations2,
    Column(period.finds_observations_directly(right)),
    Column(dataset_identifier_code.finds_observations_directly(down)),
    Column(houses.finds_observations_directly(down))
)
print("Preview tidydata2")
print(tidydata2)

## Concatenation via '+'

The simplest way to concatenate two TidyData objects is just to add them together. 

In [ ]:
tidy_data_all = tidydata1+tidydata2
print(tidy_data_all)

## Concatenation via a list

The next method is to use the `TidyData.from_tidy_list()` constructor.

In [ ]:
tidy_data_all = TidyData.from_tidy_list([tidydata1, tidydata2])
print(tidy_data_all)

## Concatenation via arguments

Lastly, you can pass in objects of class TidyData as positional arguments to the `TidyData.from_tidy()` constructor.

In [ ]:
tidy_data_all = TidyData.from_tidy(tidydata1, tidydata2)
print(tidy_data_all)

## Misaligned Headers Error

Where concatenating together TidyData the column headers must match precisely.

The following is an example of what happens if you try and concatenate two TidyData classes with misaligned headers.

In [ ]:
# We're going to redefine tidydata2 but this time we're going to label the observations column
# such that it no longer matches tidydata1 

tidydata2 = TidyData(
    observations2.label_as("Value"),
    Column(period.finds_observations_directly(right)),
    Column(dataset_identifier_code.finds_observations_directly(down)),
    Column(houses.finds_observations_directly(down))
)

# And we'll preview both so you can see the column headers no longer match
print("Preview tidydata1")
print(tidydata1)

print("Preview tidydata2")
print(tidydata2)


In [ ]:
# Now we try and concatenate them - and get an error!

tidydata1 + tidydata2